In [ ]:
import json
import requests

In [ ]:
# User informations.

user = {'username': 'admin', 'mail': 'admin@parking-system.it', 'password': 'admin', 'token': ''}

In [ ]:
# HTTP methods for the server.

def login(mail, pwd):
    url = "http://localhost:8080/login"
    headers = {"cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.get(url, headers={}, auth=(mail, pwd))

    # set token to user
    user["token"] = response.json()["token"]

    return response.json()

def get_cars_inside():
    url = "http://localhost:8080/cars"
    params = {"inside": "true"}
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.get(url=url, params=params, headers=headers)

    return response.json()

def insert_car(plate):
    url = "http://localhost:8080/cars"
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}
    body = {"plate": plate}

    response = requests.post(url=url, headers=headers, data=json.dumps(body))

    return response.json()

def make_payment(plate):
    url = "http://localhost:8080/payment" + "/" + plate
    headers = {"cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.post(url=url, headers=headers)

    return response.json()

def remove_car(plate):
    url = "http://localhost:8080/cars" + "/" + plate
    headers = {"Authorization": "Bearer " + user["token"], "cache-control": "no-cache", "Content-Type": "application/json"}

    response = requests.patch(url=url, headers=headers, data=json.dumps(body))

    return response.json()

In [ ]:
login(user['mail'], user['password'])

In [ ]:
insert_car("asfddf")

In [ ]:
make_payment("asfddf")

In [ ]:
remove_car("asfddf")

In [ ]:
get_cars_inside()

In [ ]:
##################################################################################################

In [ ]:
# Call when completely done to release memory EXIT
alpr.unload()

In [ ]:
from openalpr import Alpr
import sys

# You may want to change these two variables. Refer to the official guide http://doc.openalpr.com/sdk.html#id10
alpr = Alpr("eu", "/usr/local/src/openalpr/config/openalpr.conf.defaults", "/usr/local/src/openalpr/runtime_data")
if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)

alpr.set_top_n(10)
#alpr.set_default_region("md")
alpr.set_default_region("eu")



In [ ]:
def alpr_get_plate(image):
    results = alpr.recognize_file(image)
    #results

    if results["results"]:
        return (results["results"][0]["plate"], results["results"][0]["confidence"])
    else:
        return False

In [ ]:
alpr_get_plate("data/040603/P1010001.jpg")